## Download the data

In [16]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_text
!pip install keras


import os.path
if not os.path.isfile('data/vmware_ir_content.csv'):
    !pip install kaggle
    !kaggle competitions download -c vmware-zero-shot-information-retrieval
    !mkdir -p data/
    !unzip -o vmware-zero-shot-information-retrieval.zip
    !mv *.csv data/
    
import tensorflow_text
import tensorflow_hub as hub
import tensorflow as tf
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

import numpy as np

import pandas as pd
queries = pd.read_csv('data/test.csv')

# Created with 'extract_use.py' script
queries = pd.read_csv("data/test.csv")
corpus = pd.read_pickle('data/vmware_ir_content_parsed.pkl')

## Title - body pair

Intuition:

* titles are like queries
* title,body label a positive example
* the body of the least similar title in an embedding space, is a negative example
* Task: use title_vect,body_vect to classify positive from negative examples in a supervised setting
*       during inference, use the query instead of a title to score each body as going with on not going with the title/query

In [17]:
def identify_question_in_title(row):
    
    title = row['titleTag']
    if isinstance(title, str) and '?' in title:
        true_question = ''
        base_title = title.split('|')[0]
        if '-' in base_title:
            for part in base_title.split('-'):
                if '?' in part:
                    true_question = part
                    break
        else:
            true_question = base_title
        true_question = true_question.strip()
        if len(true_question) > 17 and true_question.endswith('?'):
            return true_question
    return None
    
corpus['question'] = corpus.apply(identify_question_in_title, axis=1)
corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name,question
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,ForgeRock Service Broker for VMware Tanzu | So...,authenticity_token,L4OfIONAOom_mTA9ShXlItMJGAUxY1nChie5y0hlPHKS0s...,ForgeRock Service Broker for VMware Tanzu | So...,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,What is DevSecOps? | VMware Tanzu,authenticity_token,5Incq1fozvzvWNsEzyt_YbeY-BEvHKmFW8WdNjxgSEi5m_...,What is DevSecOps?,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,What is DevSecOps?
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Wells Fargo | Customers | VMware Tanzu,authenticity_token,1zYYUL38ZpMZ8IibiCBQXIUsYCQHvg-vccZWYXUtRy43sx...,Wells Fargo | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Hazelcast IMDG Enterprise for VMware Tanzu | S...,authenticity_token,5iPbM3ys6YlBwLP3FrdaDofy6AUo5ZhftBmNqOLN71Zm9S...,Hazelcast IMDG Enterprise for VMware Tanzu | S...,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [18]:
question_use = corpus[~corpus['question'].isna()]['question'].apply(use)

positives = pd.DataFrame()
positives['question_use'] = question_use
positives['raw_text_use'] = raw_text_use
positives['question'] = corpus[~corpus['question'].isna()]['question']
positives['raw_text'] = corpus[~corpus['question'].isna()]['raw_text']
positives['raw_text_use'] = corpus[~corpus['question'].isna()]['raw_text_use']


positives['label'] = 1.0

positives

,question_use,raw_text_use,question,raw_text,label
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,1.0
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,1.0
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,1.0
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,1.0
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,1.0
...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0


In [19]:
def remove_question_from_text(row):
    question = row['question']
    return row['raw_text'].replace(question, '')

positives['raw_text_cleaned'] = positives.apply(remove_question_from_text, axis=1)
positives['raw_text_cleaned_use'] = positives['raw_text_cleaned'].apply(use)
positives

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.050687112, shape=(), dtype=floa..."
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,1.0,| VMware Tanzu\nTech Insights5 Minute Read\nS...,"((tf.Tensor(-0.053588558, shape=(), dtype=floa..."
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.053212874, shape=(), dtype=floa..."
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,1.0,| VMware Tanzu\nTech InsightsCloud native app...,"((tf.Tensor(-0.05014698, shape=(), dtype=float..."
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.05371097, shape=(), dtype=float..."
...,...,...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0,| VMware\nSolutions by Category\nVirtualizati...,"((tf.Tensor(-0.047710218, shape=(), dtype=floa..."
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0,| Storage Virtualization Software | VMware\nP...,"((tf.Tensor(-0.05098715, shape=(), dtype=float..."
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0,| Disaster Recovery in VM | VMware\nProducts\...,"((tf.Tensor(-0.04908049, shape=(), dtype=float..."
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0,| Free Hypervisor | VMware\nProducts\nvSphere...,"((tf.Tensor(-0.052931365, shape=(), dtype=floa..."


In [20]:
negative_rounds = 50

all_negs = []

for _ in range(0, negative_rounds):
    negatives = positives.copy()
    negatives['raw_text_cleaned_use'] = negatives.sample(frac=1.0)['raw_text_cleaned_use'].values
    negatives['label'] = -1.0
    all_negs.append(negatives)

In [21]:
training_set = pd.concat(all_negs + [positives])
training_set

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(0.02299708, shape=(), dtype=float3..."
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nS...,"((tf.Tensor(-0.062961735, shape=(), dtype=floa..."
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.06037193, shape=(), dtype=float..."
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,-1.0,| VMware Tanzu\nTech InsightsCloud native app...,"((tf.Tensor(-0.05292483, shape=(), dtype=float..."
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(0.044002835, shape=(), dtype=float..."
...,...,...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0,| VMware\nSolutions by Category\nVirtualizati...,"((tf.Tensor(-0.047710218, shape=(), dtype=floa..."
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0,| Storage Virtualization Software | VMware\nP...,"((tf.Tensor(-0.05098715, shape=(), dtype=float..."
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0,| Disaster Recovery in VM | VMware\nProducts\...,"((tf.Tensor(-0.04908049, shape=(), dtype=float..."
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0,| Free Hypervisor | VMware\nProducts\nvSphere...,"((tf.Tensor(-0.052931365, shape=(), dtype=floa..."


## Confirm training set behaves as expected

- All question embeddings should be identical per question
- Raw text embeddings should NOT

In [22]:
dev_sec_ops = training_set[training_set['question'] == 'What is DevSecOps?']
assert (dev_sec_ops['question_use'] == dev_sec_ops.iloc[0]['question_use']).all()
dev_sec_ops

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(0.02299708, shape=(), dtype=float3..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.0511161, shape=(), dtype=float3..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.051773768, shape=(), dtype=floa..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.055196922, shape=(), dtype=floa..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.05218842, shape=(), dtype=float..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.05092052, shape=(), dtype=float..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.060388777, shape=(), dtype=floa..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.006243703, shape=(), dtype=floa..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.05200958, shape=(), dtype=float..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.05500995, shape=(), dtype=float..."


In [23]:
def format_X(training_set):
    raw_text_use = training_set['raw_text_cleaned_use'].numpy()[0]
    question_use = training_set['question_use'].numpy()[0]
    concated = np.concatenate([question_use, raw_text_use])
    return concated
    #return np.concatenate(training_set['question_use'].numpy()[0],
    #                      training_set['raw_text_use'].numpy()[0])

training_set['X'] = training_set.apply(format_X, axis=1)
training_set

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use,X
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(0.02299708, shape=(), dtype=float3...","[0.025211554, 0.028694073, 0.02947252, 0.05354..."
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nS...,"((tf.Tensor(-0.062961735, shape=(), dtype=floa...","[0.055269662, 0.042888615, 0.03670096, 0.03087..."
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.06037193, shape=(), dtype=float...","[0.07775386, 0.044761658, 0.054449685, 0.03132..."
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,-1.0,| VMware Tanzu\nTech InsightsCloud native app...,"((tf.Tensor(-0.05292483, shape=(), dtype=float...","[0.008782897, 0.070924744, -0.0039736154, -0.0..."
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(0.044002835, shape=(), dtype=float...","[0.04013639, 0.0390353, 0.04971546, 0.03647691..."
...,...,...,...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0,| VMware\nSolutions by Category\nVirtualizati...,"((tf.Tensor(-0.047710218, shape=(), dtype=floa...","[0.046280254, 0.031247193, 0.015304046, -0.073..."
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0,| Storage Virtualization Software | VMware\nP...,"((tf.Tensor(-0.05098715, shape=(), dtype=float...","[0.030358074, 0.018951591, 0.06789018, -0.0155..."
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0,| Disaster Recovery in VM | VMware\nProducts\...,"((tf.Tensor(-0.04908049, shape=(), dtype=float...","[-0.013038688, 0.02732803, 0.06733354, -0.0581..."
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0,| Free Hypervisor | VMware\nProducts\nvSphere...,"((tf.Tensor(-0.052931365, shape=(), dtype=floa...","[-0.030024868, 0.017225867, 0.07627318, -0.075..."


## Confirm embeddings work as expected

For a given question/title - our question embeddings should still be first 512, and be identical.

In [24]:
dev_sec_ops = training_set[training_set['question'] == 'What is DevSecOps?']
assert (dev_sec_ops['question_use'] == dev_sec_ops.iloc[0]['question_use']).all()

def first_of_X(X):
    """First 512 should be identical"""
    sliced = X[:512]
    assert sliced.size == 512, f"Sliced size is {sliced.size}"
    return sliced

should_be_same = dev_sec_ops['X'].apply(first_of_X)
first_value = should_be_same.values[0]
# should be a cleaner way to do this with pandas, why doesn't the assert above work? Check later
for value in should_be_same.values:
    assert (first_value == value).all()

## Train a model with cross validation

Train a binary classifier with Keras using binary classification, use cross validation to find the best model

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

def build_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=1024, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

kfold = StratifiedKFold(n_splits=2, shuffle=True)
estimator = KerasClassifier(build_fn=build_model, epochs=10, batch_size=50, verbose=0)

results = cross_val_score(estimator, 
                          np.stack(training_set['X'].to_numpy()),
                          training_set['label'],
                          cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

<ipython-input-26-75609cee4a06>:14: DeprecationWarning:

KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.



Baseline: 97.96% (0.03%)


In [27]:
estimator.fit( np.stack(training_set['X'].to_numpy()),
               training_set['label'])

In [28]:
estimator.predict(np.stack([training_set['X'].iloc[0]]))

array([[-1.]])

In [ ]:
queries['query_use'] = queries['Query'].apply(use)
queries

In [25]:
def write_submission():
    from time import time
    timestamp = str(time()).replace('.', '')
    fname = f'data/turnbull_submission_{timestamp}.csv'
    print("Writing To: ", fname)
    submission[['QueryId', 'DocumentId']].to_csv(fname, index=False)

In [71]:
submission[['QueryId', 'DocumentId']]

,QueryId,DocumentId
0,0,https---www.vmware.com-topics-glossary-content...
1,0,https---www.vmware.com-products-vsphere-hyperv...
2,0,https---www.vmware.com-topics-glossary-content...
3,0,https---www.vmware.com-topics-glossary-content...
4,0,https---www.vmware.com-topics-glossary-content...
...,...,...
11750,2350,https---www.vmware.com-topics-glossary-content...
11751,2350,en-2021-vRealize-Automation-8-3-Using-and-Mana...
11752,2350,en-2021-vRealize-Automation-8-4-Using-and-Mana...
11753,2350,en-2021-VMware-Cloud-Assembly-services-Using-a...


In [72]:
write_submission()

Writing To:  data/turnbull_submission_1652544680901428.csv


In [57]:
corpus[corpus['id'].str.contains('https---blogs.vmware.com-cloudprovider-2015-11-simplifying-cloud-spending-with-vmware-subscription-purchase-program.txt')]

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
11041,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Simplifying Cloud Spending with VMware Subscri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
